# Setting up Logistic Regression for the VGG16 extracted features

In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_validate,GridSearchCV
from time import time
from Loader import train_data, test_data

In [22]:
df_real = pd.read_csv("Real_Train_Features.csv")
df_real.head(3)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,class
0,0,27.489744,0.0,0.0,0.000000,0.0,4.406033,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,3.239264,5.528229,0.0,0.0,0.0,Real
1,1,0.000000,0.0,0.0,27.298634,0.0,21.718521,17.45442,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,22.730530,0.0,0.0,0.0,Real
2,2,0.000000,0.0,0.0,0.000000,0.0,7.493243,0.00000,1.344108,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Real


In [23]:
df_fake = pd.read_csv("Fake_Train_Features.csv")
df_fake.head(3)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,class
0,0,0.000000,14.317362,0.0,29.562464,0.000000,39.905197,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,Fake
1,1,0.000000,0.000000,0.0,0.000000,10.574828,22.795412,0.0,0.000000,0.0,...,15.529102,0.0,0.0,0.0,0.0,29.724674,0.0,0.000000,18.845003,Fake
2,2,6.629343,0.000000,0.0,0.000000,13.008339,0.000000,0.0,41.318874,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1.707935,0.000000,Fake


In [24]:
df = pd.concat([df_real,df_fake],axis=0)
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,class
0,0,27.489744,0.0,0.000000,0.000000,0.000000,4.406033,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,3.239264,5.528229,0.0,0.000000,0.000000,Real
1,1,0.000000,0.0,0.000000,27.298634,0.000000,21.718521,17.454420,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,22.730530,0.0,0.000000,0.000000,Real
2,2,0.000000,0.0,0.000000,0.000000,0.000000,7.493243,0.000000,1.344108,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,Real
3,3,0.000000,0.0,29.934219,39.067970,0.000000,24.231049,6.911931,0.000000,0.0,...,16.403683,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,17.574890,Real
4,4,0.000000,0.0,0.000000,24.634619,26.340034,0.000000,0.000000,0.000000,0.0,...,13.876423,0.000000,0.0,0.0,0.000000,28.267170,0.0,0.000000,0.000000,Real
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0.000000,0.0,0.000000,4.518483,0.723795,17.608261,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,Fake
49996,49996,0.000000,0.0,0.000000,0.000000,10.060472,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.049669,0.000000,Fake
49997,49997,3.222712,0.0,0.000000,0.000000,22.387304,53.118996,0.000000,0.000000,0.0,...,0.000000,6.031562,0.0,0.0,0.000000,0.000000,0.0,0.000000,12.272759,Fake
49998,49998,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,11.401345,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,3.337986,Fake


In [25]:
df = df.iloc[:,1:]
df

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,class
0,27.489744,0.0,0.000000,0.000000,0.000000,4.406033,0.000000,0.000000,0.0,9.508491,...,0.000000,0.000000,0.0,0.0,3.239264,5.528229,0.0,0.000000,0.000000,Real
1,0.000000,0.0,0.000000,27.298634,0.000000,21.718521,17.454420,0.000000,0.0,14.870244,...,0.000000,0.000000,0.0,0.0,0.000000,22.730530,0.0,0.000000,0.000000,Real
2,0.000000,0.0,0.000000,0.000000,0.000000,7.493243,0.000000,1.344108,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,Real
3,0.000000,0.0,29.934219,39.067970,0.000000,24.231049,6.911931,0.000000,0.0,0.000000,...,16.403683,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,17.574890,Real
4,0.000000,0.0,0.000000,24.634619,26.340034,0.000000,0.000000,0.000000,0.0,0.000000,...,13.876423,0.000000,0.0,0.0,0.000000,28.267170,0.0,0.000000,0.000000,Real
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.000000,0.0,0.000000,4.518483,0.723795,17.608261,0.000000,0.000000,0.0,12.797350,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,Fake
49996,0.000000,0.0,0.000000,0.000000,10.060472,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.049669,0.000000,Fake
49997,3.222712,0.0,0.000000,0.000000,22.387304,53.118996,0.000000,0.000000,0.0,0.000000,...,0.000000,6.031562,0.0,0.0,0.000000,0.000000,0.0,0.000000,12.272759,Fake
49998,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,11.401345,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,3.337986,Fake


In [26]:
df_sample = df.sample(frac=1.0, random_state=0)

In [27]:
X_train = df_sample.drop('class', axis=1)
stnd = StandardScaler()
stnd.fit(X)
X_train = stnd.transform(X_train)
y_train = df_sample['class']

C:\Users\millerd7\Anaconda3\envs\tf2\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [31]:
def test_data_vgg16():
    df_real = pd.read_csv("/Users/millerd7/Documents/GitHub/MA415_AI_Real_Images/REAL_Test_Features.csv")
    df_fake = pd.read_csv("/Users/millerd7/Documents/GitHub/MA415_AI_Real_Images/FAKE_Test_Features.csv")
    
    df = pd.concat([df_real,df_fake],axis=0).drop('Unnamed: 0', axis=1)
    X = df.drop('class', axis=1)
    
    #le = LabelEncoder()
    y = df['class']
   # y = pd.DataFrame(le.fit_transform(y), columns=['class'])  # encode y to be numeric
    
    return (X, y)

In [32]:
X_test,y_test = test_data_vgg16()

In [33]:
time_start = time()
lgr = LogisticRegression(n_jobs = -1)
cv_results = cross_validate(lgr, X_train, y_train, return_train_score=True, cv=5)

print('Train Accuracy:', cv_results['train_score'].mean())
print('Validation Accuracy:', cv_results['test_score'].mean())
lgr.fit(X_train,y_train)
print('Test Accuracy:', lgr.score(X_test,y_test))


time_stop = time()
print('Comp Time:', round((time_stop-time_start)/60,3), 'min')

Train Accuracy: 0.8516874999999999
Validation Accuracy: 0.8478299999999999
Test Accuracy: 0.8455
Comp Time: 11.027 min


C:\Users\millerd7\Anaconda3\envs\tf2\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [36]:
time_start = time()

reg_stren = np.linspace(0.01, 0.1, 10)
print(reg_stren)
grid = {'C':reg_stren}

lgr = LogisticRegression(n_jobs = -1)
lgrCV = GridSearchCV(lgr, param_grid=grid, return_train_score=True, n_jobs=-1)
lgrCV.fit(X_train,y_train)

lgr1 = LogisticRegression(C=lgrCV.best_params_['C'], penalty='l1',solver='saga')
lgr1.fit(X_train, y_train)

ix = lgrCV.cv_results_['mean_test_score'].argmax()
print('Train score: ', lgrCV.cv_results_['mean_train_score'][ix].round(4))
print('Valid score: ', lgrCV.cv_results_['mean_test_score'][ix].round(4))
print('Test score: ', lgr1.score(X_test,y_test).round(4))

time_stop = time()
print('Comp Time:', round((time_stop-time_start)/60,3), 'min')

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 ]


C:\Users\millerd7\Anaconda3\envs\tf2\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\millerd7\Anaconda3\envs\tf2\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Train score:  0.8518
Valid score:  0.8482
Test score:  0.8458
Comp Time: 6.597 min


In [37]:
time_start = time()

reg_stren = np.linspace(0.0001, 0.001, 10)
print(reg_stren)
grid = {'C':reg_stren}

lgr = LogisticRegression(n_jobs = -1)
lgrCV = GridSearchCV(lgr, param_grid=grid, return_train_score=True, n_jobs=-1)
lgrCV.fit(X_train,y_train)

lgr2 = LogisticRegression(C=lgrCV.best_params_['C'], penalty='l2',solver='saga')
lgr2.fit(X_train, y_train)

ix = lgrCV.cv_results_['mean_test_score'].argmax()
print('Train score: ', lgrCV.cv_results_['mean_train_score'][ix].round(4))
print('Valid score: ', lgrCV.cv_results_['mean_test_score'][ix].round(4))
print('Test score: ', lgr2.score(X_test,y_test).round(4))

time_stop = time()
print('Comp Time:', round((time_stop-time_start)/60,3), 'min')

[0.0001 0.0002 0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009 0.001 ]


C:\Users\millerd7\Anaconda3\envs\tf2\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\millerd7\Anaconda3\envs\tf2\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Train score:  0.8517
Valid score:  0.8481
Test score:  0.8462
Comp Time: 6.178 min
